# Basics 
Binary Search Tree have O(n) time for lookup, add and delete if implemented naively.
But through some manipulations,$O(\log n)$ can be achieved 


In [1]:
class BstNode:
    def __init__(self, data=None, left=None, right=None):
        self.data, self.left, self.right = data, left, right

Finding maximum and minimum takes about $O(\log n)$ time for library implementations of BSTs

In [18]:
def search_bst (tree, key):
    if not tree: 
        return False
    elif key == tree.data:
        return True
    elif key < tree.data:
        return search_bst (tree.left, key)
    else:
        return search_bst (tree.right, key)
# Consicely Can be written as 
# But it's too convoluted. 

#return (tree if not tree or tree.data == key else (search_bst (
#tree.left,key) if key < tree.data else search_bst (tree.right, key)))

**Q1**. Write a program that takes as input a binary tree and checks if the tree satisfies the balanced BST property


In [19]:
def is_bst (tree): # -> Bool
    '''
    Space complexity O(1) (If use a hash table, then O(n))
    Time complexity O(n * n) (Can use a hash table to reduce the time complexity to O(n))
    *****Because you have to run is_bst for everysingle node in the tree. *****
    ''' 
    
    # O(n) As you visit all the nodes 
    def get_bst_height(tree):
        # Gets the height of the tree
        if not tree:
            return 0 
        else:
            return 1 + max (get_bst_height(tree.left), get_bst_height(tree.right))
    
    if not tree:
        return True
    else:
        return abs (get_bst_height(tree.left) - get_bst_height(tree.right)) <= 1
    


In [3]:
import collections 

def is_bst (tree):
    '''
    Utilize the fact that we do not need to know the previous results. We just need to know 
    if the previous ones are balanced & height
    '''
    BalancedStatusWithHeight = collections.namedtuple('BalancedStatusWithHeight', ('balanced','height'))
    
    def checkBalance (tree):
        if not tree:
            return BalancedStatusWithHeight(True,0)
        
        left_result = checkBalance(tree)
        if left_result.balanced == False:
            return False
        
        right_result = checkBalance(tree)
        if right_result.balanced == False:
            return False
        
        isbalanced = abs (checkBalance(tree.left).height - checkBalance(tree.right).height) <= 1
        height = max (left_result.height, right_result.height) + 1
        
        return BalancedStatusWithHeight(isbalanced, height)
    
    if not tree:
        return True
    else:
        return abs (checkBalance(tree.left).height - checkBalance(tree.right).height) <= 1 
    


Above is an example of post order travesal in the form of 
```
Algorithm Postorder(tree)
   1. Traverse the left subtree, i.e., call Postorder(left-subtree)
   2. Traverse the right subtree, i.e., call Postorder(right-subtree)
   3. Visit the root.

```
The stack is bounded by the height of the tree $O(h)$ 

**Q2** Write a program to determine the LCA (Lowest Common Ancestor) where the node does not have a parent node field

In [2]:
# Brute Force, Start from each node, runs if the this node contains node1 and node2 as children. 

def get_lca (tree, node1, node2):
'''
A rather bad approach with O(n^2)
'''    
    def contains_nodes (tree, node):
        if tree.data == node:
            return True
        else:
            return contains_node(tree.left, node) or contains_node(tree.right, node)
   
    if not tree:
        return False
    
    left_res = contains_node (tree.left, node1) and contains_node (tree.left, node2)
    right_res = contains_node (tree.right, node1)  and contains_node (tree.right, node2)
    
    if left_res:
        get_lca(tree.left, node1, node2)
        
    elif right_res:
        get_lca(tree.right, node1, node2)
        
    return tree

This uses a similar approach to keep track of the results using a status.
This time, the status tells you the number of number_of_nodes found and a **partial answer**.
You use the number of nodes found and the partial answer to answer the final question. 

In [6]:
def get_lca2 (tree, node1, node2):
    
    Status = collections.namedtuple ('Status', ('num_target_nodes', 'ancestor'))
    
    def lca_helper (tree, node1, node2): # returns status 
        
        if not tree:
            return Status (0, None)
        
        # Post Oroder Traversal
        left_res = lca_helper (tree.left, node1, node2)
        if left_res.num_target_nodes == 2: 
            return left_res # contains the result 
        
        right_res = lca_helper (tree.right, node1, node2)
        if right_res.num_target_nodes == 2:
            return right_res 
        
        
        current_total = (node1, node2).count(tree) + left_res.num_target_nodes + right_res.num_target_nodes
        if current_total == 2:
            return Status (current_total, tree)
        else:
            return Status (current_total, None)
        return Status 
            
    

**q3**  Test if a binary tree is symmetric. A symmetric tree should mirror itself from the middle, including the values of each node. 

In [1]:
def is_symmetric(tree):
    
    def helper(tree1, tree2):
        '''
        Determine if the two inputs are symmetrical
        '''
        if not tree1 and not tree2:
            return True 
        elif not tree1 or not tree2:
            return False
        elif tree1.data != tree2.data:
            return False
        else:
            return helper (tree1.left, tree2.right) and helper (tree1.right, tree2.left)
        
    
    if not tree:
        return True
    else:
        return helper(tree.left, tree.right)
        

#### Construct Binary Tree from Preoder Traversal

Return the root node of a binary search tree that matches the given preorder traversal.

(Recall that a binary search tree is a binary tree where for every node, any descendant of node.left has a value < node.val, and any descendant of node.right has a value > node.val.  Also recall that a preorder traversal displays the value of the node first, then traverses node.left, then traverses node.right.)

 
```
Example 1:

Input: [8,5,1,7,10,12]
Output: [8,5,10,1,7,null,12]
```

In [3]:
class Solution:
    def bstFromPreorder(self, preorder):
        def helper(in_left = 0, in_right = len(preorder)):
            nonlocal pre_idx
            # if there is no elements to construct subtrees
            if in_left == in_right:
                return None
            
            # pick up pre_idx element as a root
            root_val = preorder[pre_idx]
            root = TreeNode(root_val)

            # root splits inorder list
            # into left and right subtrees
            index = idx_map[root_val]

            # recursion 
            pre_idx += 1
            # build left subtree
            root.left = helper(in_left, index)
            # build right subtree
            root.right = helper(index + 1, in_right)
            return root
        
        inorder = sorted(preorder)
        # start from first preorder element
        pre_idx = 0
        # build a hashmap value -> its index
        idx_map = {val:idx for idx, val in enumerate(inorder)} 
        return helper()